<a href="https://colab.research.google.com/github/markus-hoanglong/test-gitpage/blob/master/instant_ngp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !apt update
# !apt install python3.10-dev

In [ ]:
!apt update
!apt-get install \
    build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

In [19]:
%cd ../
!mkdir nvlabs

/content


In [29]:
!ls -la ./nvlabs/

total 8
drwxr-xr-x 2 root root 4096 Nov 28 19:06 .
drwxr-xr-x 1 root root 4096 Nov 28 19:06 ..


In [ ]:
# %cd nvlabs
!rm -rf instant-ngp
!ls -la
!git clone --recursive https://github.com/NVlabs/instant-ngp.git

In [37]:
%cd instant-ngp/

/content/nvlabs/instant-ngp


In [38]:
!ls -la

total 140
drwxr-xr-x 14 root root  4096 Nov 28 19:17 .
drwxr-xr-x  3 root root  4096 Nov 28 19:16 ..
-rw-r--r--  1 root root  1849 Nov 28 19:17 .clang-format
drwxr-xr-x  2 root root  4096 Nov 28 19:17 cmake
-rw-r--r--  1 root root 15334 Nov 28 19:17 CMakeLists.txt
drwxr-xr-x  6 root root  4096 Nov 28 19:17 configs
drwxr-xr-x  5 root root  4096 Nov 28 19:17 data
drwxr-xr-x 23 root root  4096 Nov 28 19:17 dependencies
drwxr-xr-x  2 root root  4096 Nov 28 19:17 .devcontainer
drwxr-xr-x  4 root root  4096 Nov 28 19:17 docs
-rw-r--r--  1 root root   251 Nov 28 19:17 .editorconfig
-rw-r--r--  1 root root    55 Nov 28 19:17 .envrc
-rw-r--r--  1 root root  1497 Nov 28 19:17 flake.lock
-rw-r--r--  1 root root  2074 Nov 28 19:17 flake.nix
drwxr-xr-x  9 root root  4096 Nov 28 19:17 .git
-rw-r--r--  1 root root    31 Nov 28 19:17 .gitattributes
drwxr-xr-x  3 root root  4096 Nov 28 19:17 .github
-rw-r--r--  1 root root   219 Nov 28 19:17 .gitignore
-rw-r--r--  1 root root  1045 Nov 28 19:17 .gitmod

In [ ]:
!cmake . -B build -DCMAKE_BUILD_TYPE=RelWithDebInfo
!cmake --build build --config RelWithDebInfo -j

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.5.82 with host compiler GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Obtained CUDA architectures automatically from installed GPUs
-- Targeting CUDA architectures: 75
-- Module support is disabled.
-- Version: 9.0.0
-- Build type: RelWithDebInfo
-- C

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os

scene_path = "/content/gdrive/MyDrive/fox" #@param {type:"string"}
train_path = "/content/train"
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)
!rm -rf {train_path}
!mkdir {train_path}
!cp -r {scene_path}/. /content/train
transforms_path = os.path.join(train_path, f"transforms.json")
!python ./scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 4 --images {train_path} --out {transforms_path}

train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(train_path, f"{train_steps}.msgpack")
!python ./scripts/run.py --scene {train_path} --mode nerf --n_steps {train_steps} --save_snapshot {snapshot_path}